In [181]:
from __future__ import division
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os
import numbeo_scraper as ns
import get_bea_data as gbd
import population_cleanup as pc
%matplotlib inline

In [294]:
# read in population (1790 - 2010) and rj metrics meetup info (2013-2014) and merge df's

pop_df = pc.get_pop_data('data/1790-2010_MASTER.csv')
rj_df = pc.get_rj_data('data/rj_metrics.txt')
new_df = pd.concat([pop_df, rj_df], axis=1)


In [295]:
# clean and join bureau of economic affairs info

raw_bea = gbd.get_bea_data('http://www.bea.gov/newsreleases/regional/gdp_metro/2015/xls/gdp_metro0915.xls')
bea_df = gbd.clean_me(raw_bea)
bea_df = bea_df[:-2]
next_df = pd.concat([new_df, bea_df[bea_df['bea_2014'] > 20000]], axis=1)
# bea_df.shape
# cities = next_df.index


In [323]:
# incorporate numbeo data:

url_prefix = 'http://www.numbeo.com/cost-of-living/region_rankings.jsp?title='
url_suffix = '&region=021'
year_list = ['2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016']

urls = ns.build_urls(year_list)
#soup_can = [get_pages(url) for url in urls]
for url in urls:
    soup_can = ns.get_pages(url)
table_list = [ns.clean_up(soup) for soup in soup_can]
zipped = list(zip(year_list, table_list))
df_dict = ns.build_data_frames(zipped)

for item in year_list:
    columns= ns.fix_em(['Rank','City','Cost of Living Index','Rent Index','Cost of Living Plus Rent Index',
          'Groceries Index','Restaurant Price Index','Local Purchasing Power Index'])
    first_cols = columns[:2]
    first_cols.extend([column + '_{}'.format(item)for column in columns[2:]]) 
    df_dict[item].columns = first_cols    


In [324]:
def clean_up_df(df):
    df['state'] = df['city'].apply(lambda x: x.split(',')[1].strip().lower().replace(' ', '_'))
    df['city'] = df['city'].apply(lambda x: x.split(',')[0].lower().replace(' ', '_'))
    del df['rank']
    return df

In [325]:
df_2009 = clean_up_df(df_dict['2009'])
df_2010 = clean_up_df(df_dict['2010'])
df_2011 = clean_up_df(df_dict['2011'])
df_2012 = clean_up_df(df_dict['2012'])
df_2013 = clean_up_df(df_dict['2013'])
df_2014 = clean_up_df(df_dict['2014'])
df_2015 = clean_up_df(df_dict['2015'])
df_2016 = clean_up_df(df_dict['2016'])

In [329]:
df_list = [df_2009, df_2010, df_2011, df_2012, df_2013, df_2014, df_2015, df_2016]


8

In [332]:
def merger(df1, df2):
    df_merge = pd.merge(df1, df2,
              left_on=['city', 'state'],
              right_on=['city', 'state'],
              how='outer')
    return df_merge

merged1 = merger(df_list[0], df_list[1])
merged2 = merger(merged1, df_list[2])
merged3 = merger(merged2, df_list[3])
merged4 = merger(merged2, df_list[4])
merged5 = merger(merged2, df_list[5])
merged6 = merger(merged2, df_list[6])
merged7 = merger(merged2, df_list[7])
merged7.set_index('city', inplace=True)
value_list = ['canada', 'bermuda']
merged7 = merged7[~merged7['state'].isin(value_list)]

In [336]:
merged7

,cost_of_living_index_2009,rent_index_2009,cost_of_living_plus_rent_index_2009,groceries_index_2009,restaurant_price_index_2009,local_purchasing_power_index_2009,state,cost_of_living_index_2010,rent_index_2010,cost_of_living_plus_rent_index_2010,...,cost_of_living_plus_rent_index_2011,groceries_index_2011,restaurant_price_index_2011,local_purchasing_power_index_2011,cost_of_living_index_2016,rent_index_2016,cost_of_living_plus_rent_index_2016,groceries_index_2016,restaurant_price_index_2016,local_purchasing_power_index_2016
city,,,,,,,,,,,,,,,,,,,,,
san_francisco,100.14,117.65,107.09,117.42,105.24,101.50,ca,90.64,81.55,86.58,...,90.84,82.76,72.72,107.10,103.36,121.64,112.15,122.07,90.89,105.83
new_york,100.00,100.00,100.00,100.00,100.00,100.00,ny,100.00,100.00,100.00,...,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
tampa,NaN,NaN,NaN,NaN,NaN,NaN,fl,109.64,39.98,78.51,...,69.65,61.16,62.24,104.76,72.44,37.19,55.50,75.21,73.74,141.77
columbus,NaN,NaN,NaN,NaN,NaN,NaN,oh,104.69,42.63,76.96,...,NaN,NaN,NaN,NaN,69.81,34.38,52.78,71.01,64.11,133.01
portland,NaN,NaN,NaN,NaN,NaN,NaN,or,102.90,46.57,77.73,...,NaN,NaN,NaN,NaN,80.04,48.59,64.92,84.28,71.40,121.98
atlanta,NaN,NaN,NaN,NaN,NaN,NaN,ga,100.99,40.66,74.03,...,66.46,68.34,66.19,103.02,75.70,38.05,57.60,76.89,70.45,139.39
seattle,NaN,NaN,NaN,NaN,NaN,NaN,wa,100.33,49.85,77.77,...,76.52,94.13,85.61,124.94,83.22,59.97,72.05,83.63,80.14,147.45
minneapolis,NaN,NaN,NaN,NaN,NaN,NaN,mn,92.85,56.51,76.61,...,69.65,87.78,78.24,123.20,81.06,44.38,63.43,90.16,68.46,142.26
austin,NaN,NaN,NaN,NaN,NaN,NaN,tx,91.38,57.50,76.24,...,63.81,73.61,61.23,130.52,70.23,50.35,60.67,67.42,69.53,139.16


In [338]:
merged7.to_csv('/data/numbeo_complete.csv')

[autoreload of numbeo_scraper failed: Traceback (most recent call last):
  File "/Users/IXChris/anaconda/lib/python2.7/site-packages/IPython/extensions/autoreload.py", line 247, in check
    superreload(m, reload, self.old_objects)
  File "numbeo_scraper.py", line 93
    columns= ns.fix_em(['Rank','City','Cost of Living Index','Rent Index','Cost of Living Plus Rent Index', 'Groceries Index','Restaurant Price Index','Local Purchasing Power Index'])
          ^
IndentationError: expected an indented block
]


IOError: [Errno 13] Permission denied: '/data/numbeo_complete.csv'

In [47]:
url = 'https://www.biggestuscities.com/demographics/us/education-college-graduates-by-top-100-city'
# This works, don't break!'

def get_grad_data(url, cols, skip_rows=4):
    file_name = url.split('/')[-1].replace('-', '_')
    path = os.getcwd()+'/data/biggestuscities' #/{}'.format(file_name)
    file_path = '{}/{}.csv'.format(path, file_name)
    
    if not os.path.exists(path):
        os.makedirs(path)
    if not os.path.isfile(file_path):
        soup = ns.get_pages(url)
        tabs =[]
        table = soup[0].findAll('table')

        for tag in table:
            tabs.append(tag.text)

        d = [t.replace("\n",",").strip() for t in tabs][-1]
        d = d.split(' ')
        t = [item for item in d if item != '']
        t_done = [item.replace(',',' ').strip()  for item in t]
        just_table = t_done[skip_rows:-9]
        bad_item_list = ['(adsbygoogle', '=', 'window.adsbygoogle', '||', '[]).push({', '});', 'San',
                        'District', 'Of', 'North', 'St.', 'City', 'New', 'Springs', 'Urban', 'Beach',
                         'Rouge', 'Los', 'El', 'Vista', 'County', 'Fort', 'Las', 'Christi', 'Ana']

        for item in bad_item_list:
            just_table.remove(item)

        just_table = [item for item in just_table if item not in bad_item_list if item]
        just_table = [item.replace('Francisco', 'san_francisco') for item in just_table]
        just_table = [item.replace('Carolina', 'north_carolina') for item in just_table]
        just_table = [item.replace('Baton', 'baton_rouge') for item in just_table]
        just_table = [item.replace('Chula', 'chula_vista') for item in just_table]
        just_table = [item.replace('Worth', 'fort_worth') for item in just_table]
        just_table = [item.replace('Christi', 'corpus_christi') for item in just_table]
        just_table = [item.replace('Santa', 'santa_ana') for item in just_table]
        just_table = [item.replace('Vegas', 'las_vegas') for item in just_table]
        just_table = [item.replace('Bernadino', 'san_bernadino') for item in just_table]
        just_table = [item.replace('Columbia', 'dc') for item in just_table]
        just_table = [item.replace('Diego', 'san_diego') for item in just_table]
        just_table = [item.replace('Jersey', 'new_jersey') for item in just_table]
        just_table = [item.replace('Orleans', 'new_orleans') for item in just_table]
        just_table = [item.replace('York', 'new_york') for item in just_table]
        just_table = [item.lower() for item in just_table]

        chunks = [just_table[x:x+4] for x in xrange(0, len(just_table), 4)]
        df = pd.DataFrame(chunks)
        df.columns = cols
        file_name = url.split('/')[-1].replace('-', '_')
        df=df.drop('rank', axis=1)
        df.set_index('city', inplace=True)
        df_csv = df.copy()
        df_csv.to_csv(file_path)
        return df
    else:
        df = pd.read_csv(file_path)
        df.set_index('city', inplace=True)
        return df

cols = ['rank', 'city','state_bc','pct_college_grads_bc']
df = get_grad_data(url, cols)
another_df = pd.concat([next_df, df], axis=0)

#df.to_csv('/data/biggestuscities/{}'.format(filename))
another_df.shape

(592, 50)

In [263]:
url = 'https://www.biggestuscities.com/demographics/us/education-college-graduates-by-top-100-city'
cols = ['rank', 'city','state_fb','pct_foreign_born']
df2 = get_grad_data(url, cols, 6)

'/Users/IXChris/Desktop/G/capstone/data/biggestuscities/people_foreign_born_by_top_100_city.csv'

In [230]:
#subset by dense rows
meetup_df = new_df[new_df['Pop'].notnull()]
cities = list(meetup_df.index)

# fix column names
cols = meetup_df.columns
cols = [item.lower().replace(' ', '_') for item in cols]